<a href="https://colab.research.google.com/github/ricglz/CE888_activities/blob/main/assignment/Data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! [ ! -z "$COLAB_GPU" ] && pip install torch torchvision

## Preparations

Before we begin, lets mount the google drive to later on read information from it:

---



In [ ]:
from google.colab import drive

drive_path = '/content/gdrive'
drive.mount(drive_path, force_remount=False)

Mounted at /content/gdrive


## Augment the data

We'll augment the current dataset by
- Modifying the brightness of the image
- Doing random flips (vertical and horizontal)
- Doing random rotation

In [ ]:
import torchvision.transforms as T
import torchvision.datasets as datasets
from os import path

data_dir = path.join(drive_path, 'MyDrive/Flame')
resize = T.Resize(254)
transforms = T.Compose([
  resize,
  T.ColorJitter(brightness=0.5),
  T.RandomRotation(degrees=45),
  T.RandomHorizontalFlip(),
  T.RandomVerticalFlip(),
  T.ToTensor(),
])

In [ ]:
training_path = path.join(data_dir, 'Training')
train_ds = datasets.ImageFolder(training_path, transforms)

## Save the image

We want to augment only the no_fire data as it has fewer data in the class

In [ ]:
from torchvision.utils import save_image

for index, (img, label) in enumerate(train_ds):
    if label == 0:
        continue
    img_path = path.join(training_path, f'No_Fire/augmented_{index}.png')
    save_image(img, img_path)

KeyboardInterrupt: ignored